In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# 0. Lectura de datos

In [8]:
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv').values.ravel()

In [9]:
X.head()

,country_Azerbaijan,country_Other,country_T.C.,age,sex,station_Fall,station_Spring,station_Summer,station_Winter,fever_temperature,...,chronic_hematologic_disease,aids_hiv,diabetes_mellitus_type_1,diabetes_mellitus_type_2,rheumatologic_disorder,dementia,tuberculosis,smoking,other_risks,previous_positives
0,-0.088303,-0.129387,0.225631,0.233591,0.847138,-0.587958,-0.483181,-0.501048,1.353342,1.117438,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
1,-0.088303,-0.129387,0.225631,-1.092943,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-0.447312,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
2,-0.088303,-0.129387,0.225631,-0.927126,0.847138,-0.587958,-0.483181,-0.501048,1.353342,0.856647,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,0.663176
3,-0.088303,-0.129387,0.225631,0.952130,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-1.099292,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
4,-0.088303,-0.129387,0.225631,1.615396,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-0.577708,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014


División de datos

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y) # stratify=y to keep the same class distribution
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train) # stratify=y to keep the same class distribution

print('Training set:', X_train.shape)
print('Validation set:', X_val.shape)
print('Test set:', X_test.shape)

Training set: (21251, 56)
Validation set: (2362, 56)
Test set: (2624, 56)


# 1. Modelos (con las clases desbalanceadas)

En primer lugar, vamos a probar 4 modelos sin realizar ningún ajuste sobre las clases desbalanceadas, y posteriormente utilizaremos técnicas de balanceo de clases para mejorarlo. En todos los modelos aplicaremos RFE (recursive feature elimination) para seleccionar solamente las 15 características más relevantes, y posteriormente aplicaremos una búsqueda de los mejores hiperparámetros.

## 1.1 Regresión Logística 

#### RFE

In [11]:
log_reg = LogisticRegression(class_weight='balanced')
rfe = RFE(estimator=log_reg, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=LogisticRegression(class_weight='balanced'),
    n_features_to_select=15)

In [12]:
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'station_Fall',
       'station_Spring', 'station_Winter', 'history_of_fever', 'cough',
       'shortness_of_breath', 'conjunctivitis', 'fatigue_malaise', 'diarrhoea',
       'vomiting_nausea', 'smoking', 'previous_positives'],
      dtype='object')


#### Búsqueda de hiperparámetros

In [13]:
param_grid_lr = {
    'C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'penalty': ['l1','l2'],  # Penalización
}

grid_search_lr = GridSearchCV(estimator=LogisticRegression(max_iter=1000, solver='liblinear'), param_grid=param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

print("Mejores parámetros para Regresión Logística:", grid_search_lr.best_params_)

print("Accuracy en el conjunto de validación:", grid_search_lr.score(X_val[selected_features], y_val))
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_lr.predict(X_val[selected_features])))
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_lr.predict(X_val[selected_features])))

Mejores parámetros para Regresión Logística: {'C': 1, 'penalty': 'l2'}
Accuracy en el conjunto de validación: 0.8598645215918713
Matriz de confusión en el conjunto de validación:
[[  81  282]
 [  49 1950]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.62      0.22      0.33       363
           1       0.87      0.98      0.92      1999

    accuracy                           0.86      2362
   macro avg       0.75      0.60      0.63      2362
weighted avg       0.84      0.86      0.83      2362



## 1.2 Random Forest

#### RFE

In [14]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=15)

In [15]:
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_T.C.', 'age', 'sex', 'station_Fall', 'station_Spring',
       'fever_temperature', 'oxygen_saturation', 'history_of_fever', 'cough',
       'sore_throat', 'headache', 'fatigue_malaise', 'muscle_aches',
       'diarrhoea', 'previous_positives'],
      dtype='object')


#### Búsqueda de hiperparámetros

In [16]:
param_grid = {
    'n_estimators': [100, 200, 300], # Número de árboles
    'max_depth': [30, 50, 70], # Profundidad máxima
    'min_samples_split': [5, 10], # Número mínimo de muestras para dividir un nodo
    'min_samples_leaf': [2, 5], # Número mínimo de muestras en un nodo hoja
    'max_features': ['sqrt', None] # Número máximo de características a considerar en cada split
}

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

print("Accuracy en el conjunto de validación:", grid_search_rf.score(X_val[selected_features], y_val))
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_rf.predict(X_val[selected_features])))
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_rf.predict(X_val[selected_features])))

Mejores parámetros para Random Forest: {'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Accuracy en el conjunto de validación: 0.8763759525825572
Matriz de confusión en el conjunto de validación:
[[ 156  207]
 [  85 1914]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.65      0.43      0.52       363
           1       0.90      0.96      0.93      1999

    accuracy                           0.88      2362
   macro avg       0.77      0.69      0.72      2362
weighted avg       0.86      0.88      0.87      2362



## 1.3 SVM

#### RFE

In [17]:
svc = SVC(kernel='linear')
rfe = RFE(estimator=svc, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=SVC(kernel='linear'), n_features_to_select=15)

In [18]:
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'sex',
       'station_Spring', 'station_Summer', 'station_Winter',
       'fever_temperature', 'runny_nose', 'shortness_of_breath', 'chest_pain',
       'headache', 'muscle_aches', 'joint_pain', 'vomiting_nausea'],
      dtype='object')


#### Búsqueda de hiperparámetros

In [19]:
param_grid_svm = {
    'C': [0.1, 1],  # Parámetro de regularización
    'gamma': [1, 0.1, 0.01],  # Coeficiente del kernel
    'kernel': ['rbf']  # Kernel
}

grid_search_svm = GridSearchCV(estimator=SVC(class_weight='balanced'), param_grid=param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

print("Mejores parámetros para SVM:", grid_search_svm.best_params_)

print("Accuracy en el conjunto de validación:", grid_search_svm.score(X_val[selected_features], y_val))
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_svm.predict(X_val[selected_features])))
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_svm.predict(X_val[selected_features])))

Mejores parámetros para SVM: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy en el conjunto de validación: 0.7722269263336156
Matriz de confusión en el conjunto de validación:
[[ 217  146]
 [ 392 1607]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.36      0.60      0.45       363
           1       0.92      0.80      0.86      1999

    accuracy                           0.77      2362
   macro avg       0.64      0.70      0.65      2362
weighted avg       0.83      0.77      0.79      2362



# 1.4 XGBoost

#### RFE

In [20]:
xgb = XGBClassifier(random_state=42)
rfe = RFE(estimator=xgb, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=None, device=None,
                            early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, gamma=None, grow_policy=None,
                            importance_type=None, interaction_constraints=None,
                            learning_rate=None, max_bin=None,
                            max_cat_threshold=None, max_cat_to_onehot=None,
                            max_delta_step=None, max_depth=None,
                            max_leaves=None, min_child_weight=None, missing=nan,
                            monotone_constraints=None, multi_strategy=None,
                            n_estimators=None, n_jobs=None,
                            num_parallel_tree=None, random_state=42, ...),
    n_features_to_select=15)

In [21]:
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'station_Fall',
       'station_Spring', 'station_Summer', 'station_Winter',
       'history_of_fever', 'cough', 'sore_throat', 'shortness_of_breath',
       'fatigue_malaise', 'muscle_aches', 'vomiting_nausea', 'smoking'],
      dtype='object')


#### Búsqueda de hiperparámetros

In [22]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300], # Número de árboles
    'max_depth': [3, 5, 7], # Profundidad máxima
    'learning_rate': [0.1, 0.01] # Tasa de aprendizaje
}

grid_search_xgb = GridSearchCV(estimator=XGBClassifier(random_state=42), param_grid=param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train[selected_features], y_train)  # Utilizando las características

print("Mejores parámetros para XGBoost:", grid_search_xgb.best_params_)

print("Accuracy en el conjunto de validación:", grid_search_xgb.score(X_val[selected_features], y_val))
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_xgb.predict(X_val[selected_features])))
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_xgb.predict(X_val[selected_features])))

Mejores parámetros para XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
Accuracy en el conjunto de validación: 0.880186282811177
Matriz de confusión en el conjunto de validación:
[[ 179  184]
 [  99 1900]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.64      0.49      0.56       363
           1       0.91      0.95      0.93      1999

    accuracy                           0.88      2362
   macro avg       0.78      0.72      0.74      2362
weighted avg       0.87      0.88      0.87      2362



# 2. Modelos (con Oversampling en la clase 0)

En este caso vamos a aplicar los mismos 4 modelos que en el paso anterior, pero aplicando le técnica SMOTE de oversampling para balancear las clases, pues en muchos modelos anteriores observamos que el recall en la clase 0 es muy bajo, y uno de nuestros principales objetivos es reducir esa métrica.

In [23]:
print("DISTRIBUCIÓN INICIAL")
pd.Series(y_train).value_counts()

DISTRIBUCIÓN INICIAL


1    17990
0     3261
Name: count, dtype: int64

Como vemos, las clases están claramente desbalanceados, por lo que los modelos favorecen a la clase 1. Como decíamos, aplicaremos SMOTE.

In [24]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(sampling_strategy=0.8, random_state=42)

X_oversampled, y_oversampled = oversample.fit_resample(X_train, y_train)

In [ ]:
print("DISTRIBUCIÓN TRAS OVERSAMPLING")
pd.Series(y_oversampled).value_counts()

DISTRIBUCIÓN TRAS OVERSAMPLING


1    17990
0     3261
Name: count, dtype: int64

In [26]:
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(
    X_oversampled, y_oversampled, test_size=0.1, stratify=y_oversampled, random_state=42
)

print('Training set:', X_train_res.shape)
print('Test set:', X_test_res.shape)

Training set: (29143, 56)
Test set: (3239, 56)


## 2.1 Regresión Logística

#### RFE

In [27]:
log_reg = LogisticRegression(max_iter=1000, solver='liblinear')
rfe = RFE(estimator=log_reg, n_features_to_select=15)
rfe.fit(X_train_res, y_train_res)

RFE(estimator=LogisticRegression(max_iter=1000, solver='liblinear'),
    n_features_to_select=15)

In [28]:
selected_features = X_train_res.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'station_Fall',
       'station_Spring', 'station_Winter', 'oxygen_saturation',
       'history_of_fever', 'cough', 'runny_nose', 'shortness_of_breath',
       'conjunctivitis', 'fatigue_malaise', 'asthma', 'smoking'],
      dtype='object')


In [29]:
param_grid_lr = {
    'C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'penalty': ['l1','l2'],  # Penalización
}

grid_search_lr = GridSearchCV(estimator=log_reg, param_grid=param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train_res[selected_features], y_train_res)  # Utilizando las características seleccion

print("Mejores parámetros para Regresión Logística:", grid_search_lr.best_params_)

print("Accuracy en el conjunto de test:", grid_search_lr.score(X_test_res[selected_features], y_test_res))
print("Matriz de confusión en el conjunto de test:")
print(confusion_matrix(y_test_res, grid_search_lr.predict(X_test_res[selected_features])))
print("Reporte de clasificación en el conjunto de test:")
print(classification_report(y_test_res, grid_search_lr.predict(X_test_res[selected_features])))

Mejores parámetros para Regresión Logística: {'C': 10, 'penalty': 'l2'}
Accuracy en el conjunto de test: 0.7273849953689411
Matriz de confusión en el conjunto de test:
[[ 899  541]
 [ 342 1457]]
Reporte de clasificación en el conjunto de test:
              precision    recall  f1-score   support

           0       0.72      0.62      0.67      1440
           1       0.73      0.81      0.77      1799

    accuracy                           0.73      3239
   macro avg       0.73      0.72      0.72      3239
weighted avg       0.73      0.73      0.72      3239



## 2.2 Random Forest

### 2.2.1 Without grid search

#### RFE

In [30]:
rf = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=15)
rfe.fit(X_train_res, y_train_res)

RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=15)

In [31]:
selected_features = X_train_res.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_T.C.', 'age', 'sex', 'station_Fall', 'station_Spring',
       'station_Winter', 'fever_temperature', 'oxygen_saturation',
       'history_of_fever', 'cough', 'sore_throat', 'shortness_of_breath',
       'fatigue_malaise', 'muscle_aches', 'previous_positives'],
      dtype='object')


In [32]:
rf = RandomForestClassifier(random_state=42, max_depth=50,max_features=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300)
rf.fit(X_train_res[selected_features], y_train_res)

print("Accuracy en el conjunto de test:", rf.score(X_test_res[selected_features], y_test_res))
print("Matriz de confusión en el conjunto de test:")
print(confusion_matrix(y_test_res, rf.predict(X_test_res[selected_features])))
print("Reporte de clasificación en el conjunto de test:")
print(classification_report(y_test_res, rf.predict(X_test_res[selected_features])))

Accuracy en el conjunto de test: 0.8907070083359061
Matriz de confusión en el conjunto de test:
[[1264  176]
 [ 178 1621]]
Reporte de clasificación en el conjunto de test:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1440
           1       0.90      0.90      0.90      1799

    accuracy                           0.89      3239
   macro avg       0.89      0.89      0.89      3239
weighted avg       0.89      0.89      0.89      3239



### 2.2.2 With grid search

#### RFE

In [33]:
rf = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=15)
rfe.fit(X_train_res, y_train_res)

RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=15)

In [34]:
selected_features = X_train_res.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_T.C.', 'age', 'sex', 'station_Fall', 'station_Spring',
       'station_Winter', 'fever_temperature', 'oxygen_saturation',
       'history_of_fever', 'cough', 'sore_throat', 'shortness_of_breath',
       'fatigue_malaise', 'muscle_aches', 'previous_positives'],
      dtype='object')


In [35]:
param_grid = {
    'n_estimators': [100, 200, 300], # Número de árboles
    'max_depth': [30, 50, 70], # Profundidad máxima
    'min_samples_split': [5, 10], # Número mínimo de muestras para dividir un nodo
    'min_samples_leaf': [2, 5], # Número mínimo de muestras en un nodo hoja
    'max_features': ['sqrt', None] # Número máximo de características a considerar en cada split
}

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train_res[selected_features], y_train_res)  # Utilizando las características seleccionadas

print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

print("Accuracy en el conjunto de test:", grid_search_rf.score(X_test_res[selected_features], y_test_res))
print("Matriz de confusión en el conjunto de test:")
print(confusion_matrix(y_test_res, grid_search_rf.predict(X_test_res[selected_features])))
print("Reporte de clasificación en el conjunto de test:")
print(classification_report(y_test_res, grid_search_rf.predict(X_test_res[selected_features])))

Mejores parámetros para Random Forest: {'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Accuracy en el conjunto de test: 0.8934856437171966
Matriz de confusión en el conjunto de test:
[[1282  158]
 [ 187 1612]]
Reporte de clasificación en el conjunto de test:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      1440
           1       0.91      0.90      0.90      1799

    accuracy                           0.89      3239
   macro avg       0.89      0.89      0.89      3239
weighted avg       0.89      0.89      0.89      3239



## 2.3 SVC

#### RFE

In [36]:
svc = SVC(kernel='linear')
rfe = RFE(estimator=svc, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=SVC(kernel='linear'), n_features_to_select=15)

In [37]:
selected_features = X_train_res.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'sex',
       'station_Spring', 'station_Summer', 'station_Winter',
       'fever_temperature', 'runny_nose', 'shortness_of_breath', 'chest_pain',
       'headache', 'muscle_aches', 'joint_pain', 'vomiting_nausea'],
      dtype='object')


#### Búsqueda de hiperparámetros

In [38]:
param_grid_svm = {
    'C': [0.1, 1],  # Parámetro de regularización
    'gamma': [1, 0.1, 0.01],  # Coeficiente del kernel
    'kernel': ['rbf']  # Kernel
}

grid_search_svm = GridSearchCV(estimator=SVC(class_weight='balanced'), param_grid=param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train_res[selected_features], y_train_res)  # Utilizando las características seleccionadas

print("Mejores parámetros para SVM:", grid_search_svm.best_params_)

print("Accuracy en el conjunto de test:", grid_search_svm.score(X_test_res[selected_features], y_test_res))
print("Matriz de confusión en el conjunto de test:")
print(confusion_matrix(y_test_res, grid_search_svm.predict(X_test_res[selected_features])))
print("Reporte de clasificación en el conjunto de test:")
print(classification_report(y_test_res, grid_search_svm.predict(X_test_res[selected_features])))

Mejores parámetros para SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Accuracy en el conjunto de test: 0.740660697746218
Matriz de confusión en el conjunto de test:
[[ 938  502]
 [ 338 1461]]
Reporte de clasificación en el conjunto de test:
              precision    recall  f1-score   support

           0       0.74      0.65      0.69      1440
           1       0.74      0.81      0.78      1799

    accuracy                           0.74      3239
   macro avg       0.74      0.73      0.73      3239
weighted avg       0.74      0.74      0.74      3239



## 2.4 XGBoost

#### RFE

In [39]:
xgb = XGBClassifier(random_state=42)
rfe = RFE(estimator=xgb, n_features_to_select=15)
rfe.fit(X_train_res, y_train_res)

RFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=None, device=None,
                            early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, gamma=None, grow_policy=None,
                            importance_type=None, interaction_constraints=None,
                            learning_rate=None, max_bin=None,
                            max_cat_threshold=None, max_cat_to_onehot=None,
                            max_delta_step=None, max_depth=None,
                            max_leaves=None, min_child_weight=None, missing=nan,
                            monotone_constraints=None, multi_strategy=None,
                            n_estimators=None, n_jobs=None,
                            num_parallel_tree=None, random_state=42, ...),
    n_features_to_select=15)

In [40]:
selected_features = X_train_res.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_T.C.', 'station_Fall', 'station_Spring', 'station_Summer',
       'fever_temperature', 'oxygen_saturation', 'history_of_fever', 'cough',
       'sore_throat', 'shortness_of_breath', 'fatigue_malaise', 'diarrhoea',
       'asthma', 'smoking', 'previous_positives'],
      dtype='object')


In [41]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01]
}

grid_search_xgb = GridSearchCV(estimator=XGBClassifier(random_state=42), param_grid=param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train_res[selected_features], y_train_res)  # Utilizando las características

print("Mejores parámetros para XGBoost:", grid_search_xgb.best_params_)

print("Accuracy en el conjunto de test:", grid_search_xgb.score(X_test_res[selected_features], y_test_res))
print("Matriz de confusión en el conjunto de test:")
print(confusion_matrix(y_test_res, grid_search_xgb.predict(X_test_res[selected_features])))
print("Reporte de clasificación en el conjunto de test:")
print(classification_report(y_test_res, grid_search_xgb.predict(X_test_res[selected_features])))

Mejores parámetros para XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300}
Accuracy en el conjunto de test: 0.90583513430071
Matriz de confusión en el conjunto de test:
[[1269  171]
 [ 134 1665]]
Reporte de clasificación en el conjunto de test:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1440
           1       0.91      0.93      0.92      1799

    accuracy                           0.91      3239
   macro avg       0.91      0.90      0.90      3239
weighted avg       0.91      0.91      0.91      3239

